# **Generative Chatbots - GPT-based**

In  this script we show how much better are chatbots that generate responses to user inputs dynamically using machine learning models, particularly large language models (LLMs) like GPT (Generative Pre-trained Transformer). <br/>
We will use the LangChain open-source framework since it helps tremendously in the creation of these types of chatbots due to its higher-level abstractions.

## **Environment Setup**

In [1]:
import os
from getpass import getpass
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.runnables import ConfigurableFieldSpec
from langchain_community.chat_message_histories import SQLChatMessageHistory

In [2]:
# OpenAI API key
os.environ["OPENAI_API_KEY"] = getpass("Enter your OpenAI API key: ")

# Initialize OpenAI model
openai_model = ChatOpenAI(api_key=os.getenv("OPENAI_API_KEY"),
                          model="gpt-4o",
                          temperature=0,
                          max_tokens=200)

## **Stateless Chatbot**

This first chatbot has no concept of state, in other words, it has no memory of previous asked questions.

**Temperature:** The temperature parameter controls the randomness of the model's output. Lower values make the output more deterministic, while higher values introduce more creativity and variation. <br/>
**Max Tokens:** The max_tokens parameter limits the length of the response. This helps to ensure that responses are concise and focused.

### **Answer Generation and Conversation Examples**

In [3]:
def generate_response(prompt):
    response = openai_model.invoke([HumanMessage(content=prompt)])
    return response.content

In [4]:
user_input = "Can you tell me about good tourism spots in Lisbon?"
print(generate_response(user_input))

Absolutely! Lisbon, the capital of Portugal, is a city rich in history, culture, and stunning landscapes. Here are some must-visit spots:

1. **Belém Tower (Torre de Belém)**: This iconic fortress, a UNESCO World Heritage site, dates back to the 16th century and is a symbol of Portugal's Age of Discovery.

2. **Jerónimos Monastery (Mosteiro dos Jerónimos)**: Another UNESCO World Heritage site, this stunning monastery is a masterpiece of Manueline architecture and houses the tomb of the famous explorer Vasco da Gama.

3. **Alfama District**: The oldest neighborhood in Lisbon, Alfama is a maze of narrow streets, traditional Fado music venues, and historic buildings. It's a great place to wander and soak in the local atmosphere.

4. **São Jorge Castle (Castelo de São Jorge)**: Perched on a hilltop, this castle offers panoramic views of the city and the Tagus River


In [5]:
user_input = "What was my previous question?"
print(generate_response(user_input))

I'm sorry, but I don't have access to previous interactions or questions. How can I assist you today?


In [6]:
user_inputs = [
    "Hi there!",
    "Can you suggest some good restaurants in Lisbon?",
    "What about historical sites?"
]

for user_input in user_inputs:
    response = generate_response(user_input)
    print(f"User: {user_input}\nBot: {response}\n")

User: Hi there!
Bot: Hello! How can I assist you today?

User: Can you suggest some good restaurants in Lisbon?
Bot: Certainly! Lisbon is known for its vibrant culinary scene, offering a mix of traditional Portuguese cuisine and innovative modern dishes. Here are some highly recommended restaurants in Lisbon:

1. **Belcanto** - Chef José Avillez's Michelin-starred restaurant offers a sophisticated take on Portuguese cuisine. Located in Chiado, it's perfect for a special occasion.

2. **Time Out Market** - This food hall in the Mercado da Ribeira features a variety of stalls from some of Lisbon's top chefs and restaurants. It's a great place to sample different dishes in one spot.

3. **Cervejaria Ramiro** - A must-visit for seafood lovers, this bustling cervejaria is famous for its fresh shellfish and casual atmosphere.

4. **Taberna da Rua das Flores** - A cozy, traditional tavern in Chiado that serves delicious small plates (petiscos) with a modern twist. Be prepared to wait, as they

## **Context-Aware Chatbot (Stateful ChatBot)**

In [16]:
# Define the prompt template
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", 
         "You're an assistant specialized in Lisbon tourism, who speaks in {language}. Respond in 20 words or fewer."
         ),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{input}"),
    ]
)

# Runnable chain with prompt and model
runnable = prompt | openai_model

### **Memory Management**

We will use **SQLite** which is a file-based database that allows us to persist conversation history between different runs of the chatbot. This means the context can be maintained even if the application restarts, making it suitable for long-term memory.

In [12]:
def get_session_history(user_id: str, conversation_id: str):
    return SQLChatMessageHistory(f"{user_id}--{conversation_id}", "sqlite:///../DataFiles/memory.db")

with_message_history = RunnableWithMessageHistory(
    runnable,
    get_session_history,
    input_messages_key="input",
    history_messages_key="history",
    history_factory_config=[
        
        ConfigurableFieldSpec(
            id="user_id",
            annotation=str,
            name="User ID",
            description="Unique identifier for the user.",
            default="",
            is_shared=True,
        ),

        ConfigurableFieldSpec(
            id="conversation_id",
            annotation=str,
            name="Conversation ID",
            description="Unique identifier for the conversation.",
            default="",
            is_shared=True,
        ),
    ],
)

**get_session_history Function:** Returns a session history stored in an SQLite database, allowing us to track conversations based on user_id and conversation_id. <br/> 

**ConfigurableFieldSpec:** This allows us to specify unique identifiers for each user and conversation, making it possible to maintain context across different sessions.<br/>

**History Management:** The chatbot uses the stored history to provide contextually relevant responses, remembering details like the user's name and the context of the conversation.

In [10]:
# Clears the context History
def clear_session_history(user_id: str, conversation_id: str):
    history = get_session_history(user_id, conversation_id)
    history.clear()  # This clears the history for the given session

# Example:
#clear_session_history("23", "1")

### **Conversation with Context Retention**

In [17]:
# First interaction: Introduction
response = with_message_history.invoke(
    {"language": "english", "input": "Hi! I'm Ricardo, planning a trip to Lisbon."},
    config={"configurable": {"user_id": "23", "conversation_id": "1"}},
)
print("Bot:", response)

Bot: content='Hi Ricardo! Lisbon is fantastic. Be sure to visit Belém Tower, Alfama, and try the local cuisine!' response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 167, 'total_tokens': 191}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_c9aa9c0491', 'finish_reason': 'stop', 'logprobs': None} id='run-7a4de17d-b48e-4a51-a9e9-efcd8aae7fdc-0' usage_metadata={'input_tokens': 167, 'output_tokens': 24, 'total_tokens': 191}


In [19]:
# Follow-up interaction: Asking a related question
response = with_message_history.invoke(
    {"language": "english", "input": "What are some must-visit places in Lisbon?"},
    config={"configurable": {"user_id": "23", "conversation_id": "1"}},
)
print("Bot:", response)

Bot: content='Belém Tower, Jerónimos Monastery, Alfama, São Jorge Castle, and Time Out Market.' response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 255, 'total_tokens': 277}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_3aa7262c27', 'finish_reason': 'stop', 'logprobs': None} id='run-6a81f054-31ca-44a3-be3e-03221c613f35-0' usage_metadata={'input_tokens': 255, 'output_tokens': 22, 'total_tokens': 277}


In [20]:
response = with_message_history.invoke(
    {"language": "french", "input": "What are the best spots in Lisbon?"},
    config={"configurable": {"user_id": "23", "conversation_id": "1"}},
)
print("Bot:", response)

Bot: content='Miradouro da Senhora do Monte, LX Factory, Oceanário de Lisboa, and Praça do Comércio.' response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 293, 'total_tokens': 314}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_3aa7262c27', 'finish_reason': 'stop', 'logprobs': None} id='run-4b00d159-2fd8-4c6e-97ea-bb04870c1edc-0' usage_metadata={'input_tokens': 293, 'output_tokens': 21, 'total_tokens': 314}


In [22]:
# Checking if the bot remembers the user's name
response = with_message_history.invoke(
    {"language": "french", "input": "What's my name?"},
    config={"configurable": {"user_id": "23", "conversation_id": "1"}},
)
print("Bot:", response.content)

Bot: Tu t'appelles Ricardo.


In [25]:
# Checking if the bot remembers the user's name with a different user_id
response = with_message_history.invoke(
    {"language": "english", "input": "What's my name?"},
    config={"configurable": {"user_id": "20", "conversation_id": "1"}},
)
print("Bot:", response.content)

Bot: I don't know your name. Can you tell me?


We can also create a function so that we only have to change the parameters each time we ask a question 

In [26]:
def ask_chatbot(question, language="english", user_id="default_user", conversation_id="default_convo"):
    """
    :param question: The question we want to ask the chatbot.
    :param language: The language for the response (default is "english").
    :param user_id: The unique ID for the user (default is "default_user").
    :param conversation_id: The unique ID for the conversation (default is "default_convo").
    :return: The response from the chatbot.
    """
    response = with_message_history.invoke(
        {
            "language": language,
            "input": question
        },
        config={
            "configurable": {
                "user_id": user_id,
                "conversation_id": conversation_id

            }
        }
    )
    return response.content

In [27]:
response = ask_chatbot("Can you recommend some local dishes in Lisbon?", user_id="2", conversation_id="1")
print("Bot:", response)

Bot: Sure! Try Bacalhau à Brás, Pastéis de Nata, and Caldo Verde. Delicious and authentic Lisbon flavors!


In [28]:
response2 = ask_chatbot("What are some must-see attractions in Lisbon?", language="french", user_id="2", conversation_id="1")
print("Bot:", response2)

Bot: Visitez la Tour de Belém, le Monastère des Hiéronymites, l'Alfama, et l'Océanarium de Lisbonne.
